In [1]:
import folium as g

import requests
from bs4 import BeautifulSoup

import pandas as pd

## api 데이터로 그리기 - 실패

In [ ]:

geom_filter = 'POLYGON((127.990406854471 35.6978433653837,128.229287986722 35.6978433653837,128.229287986722 35.9043329483951,127.990406854471 35.9043329483951,127.990406854471 35.6978433653837))'

In [ ]:
geom_filter = 'POLYGON((127.61444091796874 35.35209603659656,127.72567749023436 35.340894535882846,127.66799926757812 35.263561862152095,127.57049560546874 35.329691482050734,127.57598876953125 35.292710387701675))'

In [ ]:
url = 'http://api.vworld.kr/req/data'
key = '46CF3A44-56C4-3D89-9E19-120DAFE16A71'
key_url = 'http://localhost:8888'
params = {
    'request':'GetFeature',
    'key' : key,
    'domain':key_url,
    'data' : 'LT_C_WGISNPGUG',
    'geomFilter':geom_filter,
    'crs' : 'EPSG:4326'
}

response = requests.get(url, params=params)

html = response.text
soup = BeautifulSoup(html, 'html.parser')

html

In [ ]:
data = response.json()['response']['result']['featureCollection']['features'][0]

In [ ]:
data

In [ ]:
park_name = data['properties']['park_name']

coordinates = data['geometry']['coordinates'][0][0]

coord = [[b, a] for a, b in coordinates]

In [ ]:
## 지도 테마 산
g_map = g.Map(location=coord[0], zoom_start=8, tiles="Stamen Terrain")
g_map

In [ ]:
g_map = g.Map(location=coord[0], zoom_start=11, tiles="cartodbpositron")

In [ ]:

# g.Polygon(locations=coord, fill=True, tooltip=park_name).add_to(g_map)
# g_map

## 좌표 json으로 지도그리기

In [2]:
def refine_coord(coord_ls):
    coord = []
    for i in coord_ls:
        a, b = i.split(' ')
        coord.append([float(b), float(a)])
    return coord

In [3]:
def make_name(df):
    name_li = df.loc['http://www.w3.org/2000/01/rdf-schema#label', f'http://lod.knps.or.kr/resource/GD_NP_PG_{file_id}']
    for i in name_li:
        if i['lang'] == 'ko':
            name = i['value'].split(' ')[0]
            break
    return name

In [ ]:
def make_json(name, boundary):
    d = {'name':name, 'polygon':boundary}
    with open(f'./data/national_park_{file_id}.json', 'w', encoding='utf-8') as file:
        file.write(json.dumps(d, ensure_ascii=False))

In [4]:
g_map = g.Map(location=[36.5477, 127.9609], zoom_start=8, tiles="cartodbpositron")
for i in range(1, 21):
    file_id = str(i).zfill(2)
    print(f'./data/GD_NP_PG_{file_id}.json')
    
    df = pd.read_json(f'./data/GD_NP_PG_{file_id}.json')
    san = df.loc['http://www.opengis.net/ont/geosparql#asWKT', f'http://lod.knps.or.kr/resource/GD_NP_PG_{file_id}']
    name = make_name(df)
    coord_ls = san[0]['value'].split('(((')[1].split('(')[0].split(')')[0].split(', ')
    boundary = refine_coord(coord_ls)
    
#     make_json(name, boundary)
    g.Polygon(locations=boundary, fill=True, tooltip=name).add_to(g_map)

./data/GD_NP_PG_01.json
./data/GD_NP_PG_02.json
./data/GD_NP_PG_03.json
./data/GD_NP_PG_04.json
./data/GD_NP_PG_05.json
./data/GD_NP_PG_06.json
./data/GD_NP_PG_07.json
./data/GD_NP_PG_08.json
./data/GD_NP_PG_09.json
./data/GD_NP_PG_10.json
./data/GD_NP_PG_11.json
./data/GD_NP_PG_12.json
./data/GD_NP_PG_13.json
./data/GD_NP_PG_14.json
./data/GD_NP_PG_15.json
./data/GD_NP_PG_16.json
./data/GD_NP_PG_17.json
./data/GD_NP_PG_18.json
./data/GD_NP_PG_19.json
./data/GD_NP_PG_20.json


In [5]:
g_map